In [17]:
import os
import cv2
import argparse
import numpy as np
import pandas as pd
import mediapipe as mp

from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

In [20]:
# define paths
SOURCE_PATH = "C:/Users/user/Desktop/수어 데이터셋/test"
DEST_PATH = "../data_preprocess/dummy_data"

In [21]:
def norm(npy):  # keypoints normalization
    scaler = MinMaxScaler()
    scaled_npy = scaler.fit_transform(npy)
    return scaled_npy    

In [25]:
hands_keypoints = []
pose_keypoints = []

for file in tqdm(os.listdir(SOURCE_PATH)):
    filename = str(os.path.splitext(file)[0])
    filepath = os.path.join(SOURCE_PATH, file)
    cap = cv2.VideoCapture(filepath)

    mp_hands = mp.solutions.hands
    mp_pose = mp.solutions.pose

    hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    while True:
        opened, image = cap.read()
        if not opened:
            break

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        hand_results = hands.process(image_rgb)
        pose_results = pose.process(image_rgb)

        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                for landmark in hand_landmarks.landmark:
                    hands_keypoints.append([landmark.x, landmark.y, landmark.z])

        if pose_results.pose_landmarks:
            for landmark in pose_results.pose_landmarks.landmark:
                pose_keypoints.append([landmark.x, landmark.y, landmark.z])

    cap.release()

    norm_hands_keypoints = norm(hands_keypoints)
    norm_pose_keypoints = norm(pose_keypoints)
    np.save(DEST_PATH + "/" + filename + "_hands.npy", norm_hands_keypoints)
    np.save(DEST_PATH + "/" + filename + "_pose.npy", norm_pose_keypoints)

hands.close()
pose.close()

  0%|          | 0/1 [00:00<?, ?it/s]c:\Users\user\anaconda3\envs\venv1\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
100%|██████████| 1/1 [00:08<00:00,  8.14s/it]
